<a href="https://colab.research.google.com/github/ruemee/Model-deployment-using-flask/blob/main/AutoInland_Vehicle_Insurance_Claim_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [39]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [40]:
link = 'https://drive.google.com/file/d/1O9PHhQkvrX9c6KiN7Ldyg9K93miFtO8i/view?usp=sharing' # The shareable link

In [41]:
linkTest='https://drive.google.com/file/d/19d-QALAYK4gFEHXKDCDnc_Aa3kKOi-Pa/view?usp=sharing'

In [42]:
linkSampleSubmission='https://drive.google.com/file/d/1w0udsBmYaBIo1UYbWZn1rQ9k3ArTgqaQ/view?usp=sharing'

In [43]:
linkHtml='https://drive.google.com/file/d/1kXzQAd-O3OZB4nmVWjyrf9nJ2nnMaGss/view?usp=sharing'

In [44]:
# to get the id part of the file 
id = link.split("/")[-2] 
  
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('Train.csv')   
  
df = pd.read_csv('Train.csv' , encoding='mac_roman',)
df.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1


In [45]:
# to get the id part of the file 
id = linkTest.split("/")[-2] 
  
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('Test.csv')   
  
test = pd.read_csv('Test.csv') 
test.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName
0,ID_009D84L,2010-04-24,2011-03-27,NO GENDER,25,2010-04-24,1,NaN,As Attached,Iveco,Victoria Island,Lagos,CVTP
1,ID_01DO2EQ,2010-01-01,2010-12-31,NaN,120,2010-01-01,4,NaN,As Attached,TOYOTA,Victoria Island,Lagos,Muuve
2,ID_01QM0NU,2010-10-23,2011-10-22,Female,46,2010-10-23,1,NaN,NaN,Ford,Abuja Municipal,Abuja-Municipal,Car Classic
3,ID_024NJLZ,2010-10-14,2011-10-13,Male,32,2010-10-14,1,NaN,NaN,NaN,Kosofe,Benue,Car Classic
4,ID_02BYET3,2010-09-16,2010-12-31,NaN,120,2010-09-16,4,NaN,NaN,TOYOTA,Victoria Island,Lagos,Muuve


In [46]:
# to get the id part of the file 
id = linkSampleSubmission.split("/")[-2] 
  
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('SampleSubmission.csv')   
   
ss = pd.read_csv('SampleSubmission.csv')

In [47]:
# # to get the id part of the file 
# id = linkHtml.split("/")[-2] 
  
# downloaded = drive.CreateFile({'id':id})  
# downloaded.GetContentFile('vehicle.html')   
   
# html = pd.read_html('vehicle.html')

In [48]:
# Check the shape of the train and test sets
print(f'The shape of the train set is: {df.shape}\nThe shape of the test set is: {test.shape}')

The shape of the train set is: (12079, 14)
The shape of the test set is: (5177, 13)


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      12079 non-null  object
 1   Policy Start Date       12079 non-null  object
 2   Policy End Date         12079 non-null  object
 3   Gender                  11720 non-null  object
 4   Age                     12079 non-null  int64 
 5   First Transaction Date  12079 non-null  object
 6   No_Pol                  12079 non-null  int64 
 7   Car_Category            8341 non-null   object
 8   Subject_Car_Colour      5117 non-null   object
 9   Subject_Car_Make        9603 non-null   object
 10  LGA_Name                5603 non-null   object
 11  State                   5591 non-null   object
 12  ProductName             12079 non-null  object
 13  target                  12079 non-null  int64 
dtypes: int64(3), object(11)
memory usage: 1.3+ MB


In [50]:
df.isnull().sum()

ID                           0
Policy Start Date            0
Policy End Date              0
Gender                     359
Age                          0
First Transaction Date       0
No_Pol                       0
Car_Category              3738
Subject_Car_Colour        6962
Subject_Car_Make          2476
LGA_Name                  6476
State                     6488
ProductName                  0
target                       0
dtype: int64

In [51]:
test.isnull().sum()

ID                           0
Policy Start Date            0
Policy End Date              0
Gender                     156
Age                          0
First Transaction Date       0
No_Pol                       0
Car_Category              1638
Subject_Car_Colour        3005
Subject_Car_Make          1061
LGA_Name                  2782
State                     2788
ProductName                  0
dtype: int64

In [52]:
# Combine train and test set
ntrain = df.shape[0] # to be used to split train and test set from the combined dataframe

all_data = pd.concat((df, test)).reset_index(drop=True)
print(f'The shape of the combined dataframe is: {all_data.shape}')

The shape of the combined dataframe is: (17256, 14)


In [53]:
# Change each column to its appriopriate datatype
date_cols = [col for col in all_data.columns if 'Date' in col]
num_cols = ['Age', 'No_Pol']
cat_cols = [col for col in all_data.columns if col not in date_cols+num_cols+['ID', 'target']]

for col in all_data.columns:
  if col in date_cols:
    all_data[col] = pd.to_datetime(all_data[col])
  elif col in cat_cols:
    all_data[col] = all_data[col].astype('category')

# Confirm whether the changes have been applied successfully
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17256 entries, 0 to 17255
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      17256 non-null  object        
 1   Policy Start Date       17256 non-null  datetime64[ns]
 2   Policy End Date         17256 non-null  datetime64[ns]
 3   Gender                  16741 non-null  category      
 4   Age                     17256 non-null  int64         
 5   First Transaction Date  17256 non-null  datetime64[ns]
 6   No_Pol                  17256 non-null  int64         
 7   Car_Category            11880 non-null  category      
 8   Subject_Car_Colour      7289 non-null   category      
 9   Subject_Car_Make        13719 non-null  category      
 10  LGA_Name                7998 non-null   category      
 11  State                   7980 non-null   category      
 12  ProductName             17256 non-null  catego

In [54]:
mapper = {'Entity':'Other', 'Joint Gender':'Other', 'NOT STATED':'Other', 'NO GENDER': 'Other', 'SEX':"Other"}
all_data.Gender = all_data.Gender.replace(mapper)

# Confirm mappings
all_data.Gender.value_counts()

Male      10888
Female     4735
Other      1118
Name: Gender, dtype: int64

In [55]:
mapper = {'Tipper Truck':'Truck'}
all_data.Car_Category = all_data.Car_Category.replace(mapper)

# Confirm mappings
all_data.Car_Category.value_counts()

Saloon                     8612
JEEP                       2868
Truck                       136
Bus                          66
Mini Bus                     55
Pick Up                      45
Motorcycle                   24
Sedan                        19
Mini Van                     18
Wagon                        13
Shape Of Vehicle Chasis       8
Station 4 Wheel               7
Van                           5
Pick Up > 3 Tons              3
CAMRY CAR HIRE                1
Name: Car_Category, dtype: int64

In [56]:
mapper = {'Mini Bus':'Bus'}
all_data.Car_Category = all_data.Car_Category.replace(mapper)

# Confirm mappings
all_data.Car_Category.value_counts()

Saloon                     8612
JEEP                       2868
Truck                       136
Bus                         121
Pick Up                      45
Motorcycle                   24
Sedan                        19
Mini Van                     18
Wagon                        13
Shape Of Vehicle Chasis       8
Station 4 Wheel               7
Van                           5
Pick Up > 3 Tons              3
CAMRY CAR HIRE                1
Name: Car_Category, dtype: int64

In [57]:
mapper = {'Mini Van':'Van'}
all_data.Car_Category = all_data.Car_Category.replace(mapper)

# Confirm mappings
all_data.Car_Category.value_counts()

Saloon                     8612
JEEP                       2868
Truck                       136
Bus                         121
Pick Up                      45
Motorcycle                   24
Van                          23
Sedan                        19
Wagon                        13
Shape Of Vehicle Chasis       8
Station 4 Wheel               7
Pick Up > 3 Tons              3
CAMRY CAR HIRE                1
Name: Car_Category, dtype: int64

In [58]:
mapper = {'Pick Up > 3 Tons':'Pick Up'}
all_data.Car_Category = all_data.Car_Category.replace(mapper)

# Confirm mappings
all_data.Car_Category.value_counts()

Saloon                     8612
JEEP                       2868
Truck                       136
Bus                         121
Pick Up                      48
Motorcycle                   24
Van                          23
Sedan                        19
Wagon                        13
Shape Of Vehicle Chasis       8
Station 4 Wheel               7
CAMRY CAR HIRE                1
Name: Car_Category, dtype: int64

In [59]:
mapper = {'Dark Gray':'Grey', 'Dark Grey': 'Grey', 'Gray & Gray':'Grey', 'Silver':'Grey', 'Gray & Silver':'Grey', 'Light Gray':'Grey','Ash':'Grey',
          'B.Silver':'Grey'}
all_data.Subject_Car_Colour = all_data.Subject_Car_Colour.replace(mapper)

In [60]:
mapper = {'Dark Blue':'Blue', 'Blue Sky': 'Blue', 'Light Blue':'Blue'}
all_data.Subject_Car_Colour = all_data.Subject_Car_Colour.replace(mapper)

In [61]:
mapper = {'D. Red':'Red', 'Burgundy': 'Red', 'Dark Red':'Red', 'Red Maroon':'Red'}
all_data.Subject_Car_Colour = all_data.Subject_Car_Colour.replace(mapper)

In [62]:
mapper = {'Light Green':'Green', 'Dark Green': 'Green'}
all_data.Subject_Car_Colour = all_data.Subject_Car_Colour.replace(mapper)

In [63]:
mapper = {'Gold':'Yellow', 'D. Gold': 'Yellow'}
all_data.Subject_Car_Colour = all_data.Subject_Car_Colour.replace(mapper)

In [64]:
mapper = {'Wine':'Cream', 'Champagne': 'Cream','Beige Mitalic':'Cream'}
all_data.Subject_Car_Colour = all_data.Subject_Car_Colour.replace(mapper)

In [65]:
# Check unique values for each categorical column
for col in cat_cols:
  print(col)
  print(all_data[col].unique(), '\n')

Gender
['Male' 'Female' 'Other' nan] 

Car_Category
['Saloon' 'JEEP' nan 'Motorcycle' 'Truck' 'Bus' 'Pick Up' 'Van'
 'CAMRY CAR HIRE' 'Wagon' 'Shape Of Vehicle Chasis' 'Sedan'
 'Station 4 Wheel'] 

Subject_Car_Colour
['Black' 'Grey' 'Red' nan 'As Attached' 'Blue' 'Green' 'White' 'Cream'
 'Brown' 'Red & Blue' 'Yellow' 'Orange' 'Red & Black' 'White & Red'
 'Purple' 'Blue & Red' 'Red & Yellow' 'Red & White' 'Black & White'
 'White & Blue' 'Blue&White&Red' 'White & Yellow' 'Black & Orange'
 'Yellow & White'] 

Subject_Car_Make
['TOYOTA', NaN, 'REXTON', 'Lexus', 'Hyundai', ..., 'BRILLIANCE', 'Buik', 'COMMANDER', 'Bajaj', 'Datsun']
Length: 76
Categories (75, object): ['TOYOTA', 'REXTON', 'Lexus', 'Hyundai', ..., 'Buik', 'COMMANDER', 'Bajaj',
                          'Datsun'] 

LGA_Name
[NaN, 'Lagos', 'Ikeja', 'Badagry', 'Eti-Osa', ..., 'Hong', 'Ifako-Agege', 'Benue', 'Okpokwu', 'Ngor-Okpala']
Length: 271
Categories (270, object): ['Lagos', 'Ikeja', 'Badagry', 'Eti-Osa', ..., 'Ifako-Agege',

In [66]:
# Fill in missing values
# For cat cols and date cols fill in with mode and for num cols fill in with 9999
for col in all_data.columns:
  if col in date_cols+cat_cols:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])
  elif col in num_cols:
    all_data[col] = all_data[col].fillna(all_data[col].fillna(9999))

# Confirm that there aren't any missing values
all_data[all_data.columns.difference(['target'])].isna().sum()

Age                       0
Car_Category              0
First Transaction Date    0
Gender                    0
ID                        0
LGA_Name                  0
No_Pol                    0
Policy End Date           0
Policy Start Date         0
ProductName               0
State                     0
Subject_Car_Colour        0
Subject_Car_Make          0
dtype: int64

In [67]:
# Extract date features from the date columns
for col in date_cols:
  for date_feature in ['year', 'month', 'day']:
    all_data[col+date_feature] = getattr(all_data[col].dt, date_feature)

all_data.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target,Policy Start Dateyear,Policy Start Datemonth,Policy Start Dateday,Policy End Dateyear,Policy End Datemonth,Policy End Dateday,First Transaction Dateyear,First Transaction Datemonth,First Transaction Dateday
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,Victoria Island,Lagos,Car Classic,0.0,2010,5,14,2011,5,13,2010,5,14
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,Victoria Island,Lagos,Car Classic,1.0,2010,11,29,2011,11,28,2010,11,29
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,Victoria Island,Lagos,Car Classic,0.0,2010,3,21,2011,3,20,2010,3,21
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,Saloon,Black,TOYOTA,Victoria Island,Lagos,CarSafe,0.0,2010,8,21,2011,8,20,2010,8,21
4,ID_00BRP63,2010-08-29,2010-12-31,Other,20,2010-08-29,3,Saloon,Black,TOYOTA,Lagos,Lagos,Muuve,1.0,2010,8,29,2010,12,31,2010,8,29


In [68]:
# all_data = all_data.drop(["State"],axis=1)

In [69]:
# Encode categorical features
all_data = pd.get_dummies(data = all_data, columns = cat_cols)
all_data.head()

,ID,Policy Start Date,Policy End Date,Age,First Transaction Date,No_Pol,target,Policy Start Dateyear,Policy Start Datemonth,Policy Start Dateday,Policy End Dateyear,Policy End Datemonth,Policy End Dateday,First Transaction Dateyear,First Transaction Datemonth,First Transaction Dateday,Gender_Female,Gender_Male,Gender_Other,Car_Category_Bus,Car_Category_CAMRY CAR HIRE,Car_Category_JEEP,Car_Category_Motorcycle,Car_Category_Pick Up,Car_Category_Saloon,Car_Category_Sedan,Car_Category_Shape Of Vehicle Chasis,Car_Category_Station 4 Wheel,Car_Category_Truck,Car_Category_Van,Car_Category_Wagon,Subject_Car_Colour_As Attached,Subject_Car_Colour_Black,Subject_Car_Colour_Black & Orange,Subject_Car_Colour_Black & White,Subject_Car_Colour_Blue,Subject_Car_Colour_Blue & Red,Subject_Car_Colour_Blue&White&Red,Subject_Car_Colour_Brown,Subject_Car_Colour_Cream,...,State_Nsit-Ubium,State_Obafemi-Owode,State_Obia-Akpor,State_Ogba-Ndoni,State_Ogbmosho-South,State_Ogun,State_Ogun-Waterside,State_Ondo,State_Ondo-West,State_Onitsha-North,State_Onitsha-South,State_Orile-Iganmu,State_Oshimili-North,State_Oshodi-Isolo,State_Osun,State_Ovia-SouthWest,State_Owerri-Municipal,State_Owerri-North,State_Owerri-West,State_Oyo,State_Oyo-East,State_Oyo-West,State_Port-Harcourt,State_QuaAn-Pan,State_Rivers,State_Udi-Agwu,State_Ughelli-North,State_Umuahia-South,State_Warri-Central,State_Warri-North,State_Warri-South,ProductName_CVTP,ProductName_Car Classic,ProductName_Car Plus,ProductName_Car Vintage,ProductName_CarFlex,ProductName_CarSafe,ProductName_Customized Motor,ProductName_Motor Cycle,ProductName_Muuve
0,ID_0040R73,2010-05-14,2011-05-13,30,2010-05-14,1,0.0,2010,5,14,2011,5,13,2010,5,14,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,ID_0046BNK,2010-11-29,2011-11-28,79,2010-11-29,1,1.0,2010,11,29,2011,11,28,2010,11,29,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,ID_005QMC3,2010-03-21,2011-03-20,43,2010-03-21,1,0.0,2010,3,21,2011,3,20,2010,3,21,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,ID_0079OHW,2010-08-21,2011-08-20,2,2010-08-21,1,0.0,2010,8,21,2011,8,20,2010,8,21,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,ID_00BRP63,2010-08-29,2010-12-31,20,2010-08-29,3,1.0,2010,8,29,2010,12,31,2010,8,29,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [70]:
# all_data = all_data.merge('Policy Start Datemonth','Policy End Datemonth')

In [71]:
all_data.head()

,ID,Policy Start Date,Policy End Date,Age,First Transaction Date,No_Pol,target,Policy Start Dateyear,Policy Start Datemonth,Policy Start Dateday,Policy End Dateyear,Policy End Datemonth,Policy End Dateday,First Transaction Dateyear,First Transaction Datemonth,First Transaction Dateday,Gender_Female,Gender_Male,Gender_Other,Car_Category_Bus,Car_Category_CAMRY CAR HIRE,Car_Category_JEEP,Car_Category_Motorcycle,Car_Category_Pick Up,Car_Category_Saloon,Car_Category_Sedan,Car_Category_Shape Of Vehicle Chasis,Car_Category_Station 4 Wheel,Car_Category_Truck,Car_Category_Van,Car_Category_Wagon,Subject_Car_Colour_As Attached,Subject_Car_Colour_Black,Subject_Car_Colour_Black & Orange,Subject_Car_Colour_Black & White,Subject_Car_Colour_Blue,Subject_Car_Colour_Blue & Red,Subject_Car_Colour_Blue&White&Red,Subject_Car_Colour_Brown,Subject_Car_Colour_Cream,...,State_Nsit-Ubium,State_Obafemi-Owode,State_Obia-Akpor,State_Ogba-Ndoni,State_Ogbmosho-South,State_Ogun,State_Ogun-Waterside,State_Ondo,State_Ondo-West,State_Onitsha-North,State_Onitsha-South,State_Orile-Iganmu,State_Oshimili-North,State_Oshodi-Isolo,State_Osun,State_Ovia-SouthWest,State_Owerri-Municipal,State_Owerri-North,State_Owerri-West,State_Oyo,State_Oyo-East,State_Oyo-West,State_Port-Harcourt,State_QuaAn-Pan,State_Rivers,State_Udi-Agwu,State_Ughelli-North,State_Umuahia-South,State_Warri-Central,State_Warri-North,State_Warri-South,ProductName_CVTP,ProductName_Car Classic,ProductName_Car Plus,ProductName_Car Vintage,ProductName_CarFlex,ProductName_CarSafe,ProductName_Customized Motor,ProductName_Motor Cycle,ProductName_Muuve
0,ID_0040R73,2010-05-14,2011-05-13,30,2010-05-14,1,0.0,2010,5,14,2011,5,13,2010,5,14,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,ID_0046BNK,2010-11-29,2011-11-28,79,2010-11-29,1,1.0,2010,11,29,2011,11,28,2010,11,29,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,ID_005QMC3,2010-03-21,2011-03-20,43,2010-03-21,1,0.0,2010,3,21,2011,3,20,2010,3,21,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,ID_0079OHW,2010-08-21,2011-08-20,2,2010-08-21,1,0.0,2010,8,21,2011,8,20,2010,8,21,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,ID_00BRP63,2010-08-29,2010-12-31,20,2010-08-29,3,1.0,2010,8,29,2010,12,31,2010,8,29,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [72]:

corr_matrix = all_data.corr()
corr_matrix["target"].sort_values(ascending=False)

target                     1.000000
ProductName_Muuve          0.156545
ProductName_Car Classic    0.149370
Subject_Car_Colour_Grey    0.108784
Subject_Car_Make_Iveco     0.091159
                             ...   
LGA_Name_Okpokwu                NaN
LGA_Name_VGC                    NaN
LGA_Name_Zaria                  NaN
State_Idemili-south             NaN
State_Ngor-Okpala               NaN
Name: target, Length: 518, dtype: float64

In [73]:
# Separate train and test data from the combined dataframe
train_df = all_data[:ntrain]
test_df = all_data[ntrain:]

# Check the shapes of the split dataset
train_df.shape, test_df.shape

((12079, 522), (5177, 522))

In [74]:
# Select main columns to be used in training
main_cols = all_data.columns.difference(date_cols+['ID', 'target','No_Pol', 'State','Policy Start Datemonth','First Transaction Dateyear','First Transaction Dateday'])
X = train_df[main_cols]
y = train_df.target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

# Train model
model = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=1000,
                                learning_rate=0.1,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)
model.fit(X_train, y_train)

# Make predictions
# y_pred = model.predict(X_test)

# Check the f1 score of the model
# print(f'F1 score on the X_test is: {f1_score(y_test, y_pred)}')

LGBMClassifier(bagging_fraction=0.9, boosting_type='gbdt', class_weight=None,
               colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=1000, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.2, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [75]:
# # Make prediction on the test set
# test_df = test_df[main_cols]
# predictions = model.predict(test_df)

# # Create a submission file
# sub_file = ss.copy()
# sub_file.target = predictions

# # Check the distribution of your predictions
# sns.countplot(sub_file.target);

In [76]:
# # Create a csv file and upload to zindi 
# sub_file.to_csv('Baseline.csv', index = False)

In [77]:
# # Saving model to current directory
# # Pickle serializes objects so they can be saved to a file, and loaded in a program again later on.
# import pickle
# pickle.dump(model, open('model.pkl','wb'))

In [78]:
#Loading model to compare the results
# model1 = pickle.load(open('model.pkl','rb'))
# print(model1.predict([[2.6, 8, 10.1]]))

In [79]:
# !pip install flask

In [80]:
!pip install flask-ngrok

In [81]:
#import libraries
import numpy as np
from flask import Flask, render_template,request
import pickle#Initialize the flask App


In [82]:
# #default page of our web-app
# @app.route('/')
# def home():
    

In [83]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template,request

from google.colab import drive
drive.mount('/content/gdrive/')

app = Flask(__name__, template_folder='/content/gdrive/MyDrive')
model = pickle.load(open('model.pkl', 'rb'))
run_with_ngrok(app)   
  
@app.route("/")
def home():
    return render_template('vehicle.html')
    


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [84]:
#To use the predict button in our web-app
@app.route('/predict',methods=['POST'])
def predict():
    #For rendering results on HTML GUI
    int_features = [float(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    prediction = model.predict(final_features)
    output = round(prediction[0], 2) 
    return render_template('vehicle.html', prediction_text='CO2    Emission of the vehicle is :{}'.format(output))

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bf7feef03780.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Jun/2021 19:01:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2021 19:01:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Jun/2021 19:01:06] "GET / HTTP/1.1" 200 -
[2021-06-02 19:02:11,405] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936